## Password Checker

In [2]:
_password = "floccinaucinihilipilification"

def naive_checker(attempt):
    n = len(_password)
    success = True
    if len(attempt) != n:
        success = False
    else:
        for idx in range(n):
            if attempt[idx] != _password[idx]:
                success = False
                break
    return success


print(naive_checker("super"))
print(naive_checker("floccinaucinihilipilification"))

%timeit -n 100 naive_checker("super")
%timeit -n 100 naive_checker("floccinaucinihilipilification")
%timeit -n 100 naive_checker("floccinaucinihiiiiiiiiiiiiiii")

False
True
131 ns ± 7.74 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.65 μs ± 37.5 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
930 ns ± 114 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
import timeit
import numpy as np

#get average time
def validation_time(attempt, n = 1, f = naive_checker):
    times = []
    for _ in range(n):
        start_time = timeit.default_timer()
        f(attempt)
        end_time = timeit.default_timer()
        times.append(end_time - start_time)
    return sum(times)/n

print(validation_time("floccinaucinihilipilification", 100))
print(validation_time("floccinaucinihiiiiiiiiiiiiiii", 100))
print(validation_time("super", 100))

1.3009999838686782e-06
9.849999878497328e-07
1.7400001524947584e-07


## Genetic Algorithm

Representation of the solution
 - Assume attempts are correct length
 - Objective Value: Time taken to get back success result
 - Population: 1000 solutions

Selection Strategy
 - Use roulette wheel to choose best solutions as parents

Reproduction Strategy
 - Crossover: Breeding ideas:
  - Preserve first run of same characters
  - TPX (Two Point Crossover)

 - Mutation: 

Replacement Strategy
- replace lambda proportion of the population

In [4]:
import random
import string

#initial population
pass_length = len(_password)
valid_charset = string.ascii_lowercase
pop_size = 500
random.seed(1)

init_population = [''.join(random.choice(valid_charset) for _ in range(pass_length)) for _ in range (pop_size)]

init_population[0] = "floccinaucinihiiiiiiiiiiiiiii"
print(init_population)


['floccinaucinihiiiiiiiiiiiiiii', 'hsdkaaauramvgnxaqhyoprhlhvhyo', 'janrudfuxjdxkxwqnqvgjjspqmsbp', 'hxzmnvflrwyvxlcovqdyfqmlpxapb', 'jwtssmuffqhaygrrhmqlsloivrtxa', 'mzxqzeqyrgnbplsrgqnplnlarrtzt', 'kotazhufrsfczrzibvccaoayyihid', 'ztfljcffiqfviuwjowkppdajmknzg', 'idixqgtnahamebxfowqvnrhuzwqoh', 'quamvszkvunbxjegbjccjjxfnsiea', 'rbsgsofywtqbmgldgsvnsgpdvmjqp', 'aktmjafgkzszekngivdmrlvrpyrhc', 'xbceffrgiyktqilkkdjhtywpesryd', 'kbncmzeekdtszmcsrhsciljsrdoid', 'zbjatvacndzbghzsnfdofvhfxdnmz', 'rjriwpkdgukbaazjxtkomkmccktod', 'igztyrwpvlifrgjghlcicyocusukh', 'mjbkfkzsjhkdrtsztchhazhmcircx', 'cauajyzlppedqyzkcqvffyeekjdwq', 'tjegerxbyktzvrxwgfjnrfbwvhiyc', 'voznriroroamkfipazunsabwlsese', 'eiimsmftchpafqkquovuxhhkpvphw', 'nkrtxuiuhbcyqulfqyzgjjwjrlfww', 'xotcdtqsmfeingsxyzbpvmwulmqfr', 'xbqcziudixceytvvwcohmznmfkoet', 'pgdntrndvjihmxragqosaauthigfj', 'ergijsyivozzfrlpndygsmgjzdzad', 'sxarjvyxuecqlszjnqvlyqkadowol', 'jrmkzxvspdummgraiutxxxqgotqnx', 'wjwfotvqglqavmsnmktsxwxcpxhuu', 'juanxueu

In [5]:
#generate timings for each attempt
def fitness(population, f = naive_checker):
    n_repeats = 1000
    timings = []
    for individual in population:
        ind_time = validation_time(individual, n_repeats, f)
        timings.append(ind_time)
    return timings

print(fitness(init_population))

[1.173800013930304e-06, 4.7419999282283244e-07, 2.4590000793978107e-07, 3.605000019888394e-07, 3.1680000211053996e-07, 2.575999988039257e-07, 2.6479999905859587e-07, 2.627999929245561e-07, 2.477000025464804e-07, 3.102999780821847e-07, 4.6949998795753345e-07, 3.7970000084897037e-07, 2.527999968151562e-07, 2.424000067549059e-07, 2.418000103716622e-07, 3.189999815731426e-07, 2.511000047888956e-07, 2.4640000719955425e-07, 2.411999948890298e-07, 2.489000062269042e-07, 2.585999973234721e-07, 2.555999899414019e-07, 2.5979999463743297e-07, 2.564999958849512e-07, 2.589999976407853e-07, 2.550999970480916e-07, 2.424000131213688e-07, 2.435999958834145e-07, 2.4459999622195026e-07, 2.449999992677476e-07, 2.5660000574134756e-07, 3.1450000915356215e-07, 3.0279999919002875e-07, 4.6940000174799935e-07, 3.1960000524122735e-07, 3.1380000109493267e-07, 3.09399997604487e-07, 3.128000016658916e-07, 3.147000034005032e-07, 3.136000050290022e-07, 3.183000108037959e-07, 3.1479998233407967e-07, 3.143999938401976e

In [6]:
#Defining Crossover and Mutation
mutation_rate = 0.02

def crossover(parent1, parent2):
    child = []
    run_same = True

    for i in range(pass_length):
        if run_same and parent1[i] == parent2[i]:
            child.append(parent1[i])
        else:
            run_same = False
            child.append(random.choice([parent1[i], parent2[i]]))
    
    return "".join(child)

def mutate(individual):
    individual = list(individual)
    
    for i in range(len(individual)):
        if random.random() < mutation_rate:  
            new_char = random.choice(valid_charset)
            # Ensure mutation produces a different character
            while new_char == individual[i]:  
                new_char = random.choice(valid_charset)
            individual[i] = new_char
    
    return ''.join(individual)

In [7]:
lambda_proportion = 0.8

def roulette_wheel_selection(population, fitness):
  
    max_fitness = max(fitness)

    '''Better to not normalise?'''
    #normalized_fitness = [max_fitness - f for f in fitness]
    
    # Calculate cumulative probabilities
    total_fitness = sum(fitness)
    probabilities = [f / total_fitness for f in fitness]
    cumulative_probs = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
    
    # Select lambda proportion of the population
    selected_count = int(len(population) * lambda_proportion)
    selected = []
    
    for _ in range(selected_count):
        r = random.random()
        for i, cum_prob in enumerate(cumulative_probs):
            if r <= cum_prob:
                selected.append(population[i])
                break
    
    return selected


def next_generation(selected):
    next_generation = []
    
    while len(next_generation) < pop_size:

        parent1, parent2 = random.sample(selected, 2)

        child = crossover(parent1, parent2)
        child = mutate(child)
        
        next_generation.append(child)
    
    return next_generation





In [ ]:
#tracking Storage
best_fitness_storage = []
avg_fitness_storage = []

def genetic_algorithm(initial_population, generations):
    population = initial_population
    fitness_scores = fitness(population)
    



    for gen in range(generations):
        print(f"Generation {gen + 1}")
        
        # Roulette wheel selection
        selected = roulette_wheel_selection(population, fitness_scores)
        
        # Generate next generation
        population = next_generation(selected)
        
        # Recalculate fitness for the new generation
        fitness_scores = fitness(population)
        
        # # Print best solution so far
        # best_fitness = max(fitness_scores)
        # best_individual = population[fitness_scores.index(best_fitness)]
        # print(f"Best Individual: {best_individual}, Fitness: {best_fitness}")
        # Calculate metrics
        best_fitness = max(fitness_scores)
        best_individual = population[fitness_scores.index(best_fitness)]
        avg_fitness = sum(fitness_scores) / len(fitness_scores)
        
        # Print metrics
        print(f"Best Fitness: {best_fitness}")
        print(f"Average Fitness: {avg_fitness}")
        print(f"Best Individual: {best_individual}")
        
        # Store metrics for plotting
        best_fitness_storage.append(best_fitness)
        avg_fitness_storage.append(avg_fitness)
 
    
    return population, fitness_scores

: 

In [ ]:
generations = 1000

final_population, final_fitness = genetic_algorithm(init_population, generations)

Generation 1
Best Fitness: 6.629000081375125e-07
Average Fitness: 2.6784999973097e-07
Best Individual: zhekfsivwagkzrbbmwgfbpgmzvwsy
Generation 2
Best Fitness: 1.1313999912090367e-06
Average Fitness: 2.7380399982939706e-07
Best Individual: abvompjzceuhokyglxfcxskgvwldu
Generation 3
Best Fitness: 9.245000073860865e-07
Average Fitness: 2.6504060047955136e-07
Best Individual: ilitkymcpqsddzmrhzjvcdollcrnl
Generation 4
Best Fitness: 7.850999982110807e-07
Average Fitness: 2.1057279992237452e-07
Best Individual: qzxqvppypbrclahrvqeplnrxprhzy
Generation 5
Best Fitness: 7.393000005322392e-07
Average Fitness: 2.2433319989431767e-07
Best Individual: sikwttiwvywkhsyyhbtmxsrznthcu
Generation 6
Best Fitness: 6.153999847811065e-07
Average Fitness: 2.2150240052906156e-07
Best Individual: zfigknogxfucabbcuvxyxbdgyurpt
Generation 7
Best Fitness: 6.52599996101344e-07
Average Fitness: 2.2656460020152735e-07
Best Individual: sdpehbifmfbxxchxhqauykimnfxfa
Generation 8
Best Fitness: 8.43000014356221e-07
Ave

In [ ]:
import matplotlib.pyplot as plt

print(final_population)
def plot_metrics(best_fitness, avg_fitness):
    generations = range(1, len(best_fitness) + 1)
    
    plt.figure(figsize=(12, 8))
    
    # Best Fitness and Average Fitness
    plt.subplot(2, 2, 1)
    #plt.plot(generations, best_fitness, label="Best Fitness", color="blue")
    plt.plot(generations, avg_fitness, label="Average Fitness", color="orange")
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.title("Fitness Over Generations")
    plt.legend()

    
    plt.tight_layout()
    plt.show()

plot_metrics(best_fitness_storage, avg_fitness_storage)

NameError: name 'final_population' is not defined

# BFS first n letters

In [ ]:
def fitness_function(attempt):
    n_repeats = 3
    
    ind_time = validation_time(attempt, n_repeats)
    return ind_time

In [ ]:
import itertools

def brute_force_best_next_letters(current_solution, password, fitness_function):
    """
    Brute force all permutations of the next 3 letters and return the best solution.
    """
    remaining_length = len(password) - len(current_solution)
    segment_length = min(3, remaining_length)  # Restrict to the next 3 letters or fewer
    
    # Generate all possible combinations for the next `segment_length` letters
    all_combinations = itertools.product(string.ascii_lowercase, repeat=segment_length)
    
    best_solution = current_solution
    best_fitness = float('-inf')  # Start with the worst possible fitness
    
    for combo in all_combinations:
        # Extend the current solution with this combination
        candidate_solution = current_solution + ''.join(combo)
        
        # Fill the rest with random guesses to match the password length
        candidate_solution += ''.join(random.choices(string.ascii_lowercase, k=(len(password) - len(candidate_solution))))
        
        # Evaluate fitness of the candidate solution
        candidate_fitness = fitness_function(candidate_solution)
        
        # Update the best solution if the candidate is better
        if candidate_fitness > best_fitness:
            best_solution = candidate_solution
            best_fitness = candidate_fitness
    
    return best_solution, best_fitness

In [ ]:
def genetic_algorithm_with_brute_force(initial_population, generations, brute_force_interval):
    population = initial_population
    fitness_scores = fitness(population)
    
    for gen in range(generations):
        print(f"\n=== Generation {gen + 1} ===")
        
        # Print the population's best individual and its fitness
        best_fitness = max(fitness_scores)
        best_individual = population[fitness_scores.index(best_fitness)]
        print(f"Current Best Individual: {best_individual}, Fitness: {best_fitness}")
        
        # Roulette wheel selection
        selected = roulette_wheel_selection(population, fitness_scores)
        
        # Generate next generation
        population = next_generation(selected)
        
        # Recalculate fitness for the new generation
        fitness_scores = fitness(population)
        
        # Brute force step every `brute_force_interval` generations
        if (gen + 1) % brute_force_interval == 0:
            print("\nRunning Brute Force Step...")
            
            # Select the top 10 individuals based on fitness
            top_indices = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i], reverse=True)[:5]
            top_solutions = [population[i] for i in top_indices]
            
            # Run brute force for the next 3 letters for each top solution
            new_population = []
            for solution in top_solutions:
                brute_forced_solution, brute_force_fitness = brute_force_best_next_letters(solution, _password, fitness_function)
                print(f"Top Solution: {solution} -> Brute Forced: {brute_forced_solution}, Fitness: {brute_force_fitness}")
                new_population.append((brute_forced_solution, brute_force_fitness))
            
            # Sort by fitness and keep the top 50 solutions
            new_population = sorted(new_population, key=lambda x: x[1], reverse=True)[:20]
            population = [ind for ind, fit in new_population]
            fitness_scores = [fit for ind, fit in new_population]
            
            print(f"Brute Force Population Updated. Best Solution: {population[0]}, Fitness: {fitness_scores[0]}")
    
    return population, fitness_scores


In [ ]:
brute_force_interval = 1000
generations = 10000

# Run the genetic algorithm with brute force
final_population, final_fitness = genetic_algorithm_with_brute_force(
    init_population, 
    generations, 
    brute_force_interval
)

# Final best solution
best_individual = final_population[final_fitness.index(max(final_fitness))]
print(f"\nFinal Best Individual: {best_individual}, Fitness: {max(final_fitness)}")